# Program pembuat model kelas 14 pake GoogleColab

sumber : https://github.com/jackpraveenraj/Stock-Prediction-Using-Stacked-LSTM/blob/main/Stock%20Prediction%20Using%20LSTM.ipynb

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LSTM
from keras.callbacks import ModelCheckpoint

In [2]:
# menghubungkan drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# mengimport file python/modul python yang ada di drive
import sys
sys.path.insert(0,'/content/drive/MyDrive/iky/fungsi')
import datanya

In [11]:
timestep = 100

# ~~~~~ file yang berisi ticker jadikan sebuah list
with open('drive/MyDrive/iky/data/ticker_issi.txt') as file:
    tick = file.readlines()
    ticker = [line.rstrip() for line in tick]# ini adalah list yang berisi ticker

In [26]:
ticker = ['APEX']#['YPAS'] # 

In [27]:
for tic in ticker:
    # df = pd.read_csv(f'drive/MyDrive/iky/data/bsaham/{tic}.csv')
    df = pd.read_csv(f'drive/MyDrive/campuran/data/saham/{tic}.csv')
    x_train, y_train = datanya.data_latih12(df, timestep)
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # filepath = f"drive/MyDrive/iky/kelas_9/model/{tic}_model.h5"
    # checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    # callbacks_list = [checkpoint]
    # model.fit(X_train, y_train, batch_size=cur_batch_size, epochs=cur_epochs, callbacks=callbacks_list,
    #           shuffle=True)
    model.fit(x_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
54/54 [==============================] - 15s 179ms/step - loss: 0.0214
Epoch 2/50
54/54 [==============================] - 10s 177ms/step - loss: 0.0035
Epoch 3/50
54/54 [==============================] - 10s 178ms/step - loss: 0.0031
Epoch 4/50
54/54 [==============================] - 10s 177ms/step - loss: 0.0028
Epoch 5/50
54/54 [==============================] - 10s 180ms/step - loss: 0.0026
Epoch 6/50
54/54 [==============================] - 10s 179ms/step - loss: 0.0028
Epoch 7/50
54/54 [==============================] - 10s 179ms/step - loss: 0.0024
Epoch 8/50
54/54 [==============================] - 10s 179ms/step - loss: 0.0027
Epoch 9/50
54/54 [==============================] - 10s 179ms/step - loss: 0.0023
Epoch 10/50
54/54 [==============================] - 10s 181ms/step - loss: 0.0023
Epoch 11/50
54/54 [==============================] - 10s 178ms/step - loss: 0.0023
Epoch 12/50
54/54 [==============================] - 10s 177ms/step - loss: 0.0022
Epoch 13/50
5

In [28]:
# ~~~~~~~~~~~~~~~~ fungsi untuk menggunakan model kelas 1 untuk memprediksi

import pandas as pd
from keras.models import load_model
from datetime import date

## fungsi untuk menghitung kenaikan harga
def kenaikan_persen(nilai_awal, nilai_terakhir):
    persen = float(((nilai_terakhir-nilai_awal)/nilai_awal)*100)
    if persen >= 0:
        akan = [1,'Naik']
    elif persen <= 0:
        akan = [0,'Turun']
    return persen, akan

def pake_k14(list_ticker, timestep=100):
    hasil_model = pd.DataFrame(columns=['Saham','Perubahan nilai','Keadaan','0/1','Perubahan(%)'])
    for tic in list_ticker:
        makanannya = pd.read_csv(f'drive/MyDrive/campuran/data/saham/{tic}.csv')
        # model = load_model(f"kelas_14/model/{tic}_model.h5")
        hasil_pred = datanya.predik_5h(makanannya, timestep, model)
        kenaikan = hasil_pred[4][0] - hasil_pred[0][0]
        persentase, akan = kenaikan_persen(hasil_pred[0][0], hasil_pred[4][0])
        hasil_model = pd.concat([hasil_model, pd.DataFrame.from_records([{
            'Saham': tic, 'Perubahan nilai' : kenaikan, 'Keadaan': akan[1],
            '0/1':akan[0],'Perubahan(%)' : persentase}])])
    # mengurutkan sesuai kolom Perubahan(%), dari nilai terbesar ke terkecil
    hasil_model.sort_values(by='Perubahan(%)',ascending=False, inplace=True) 
    hasil_model.reset_index(inplace=True, drop=True) # mereset index
    return hasil_model

In [29]:
hasil_k14 = pake_k14(ticker)
hasil_k14

,Saham,Perubahan nilai,Keadaan,0/1,Perubahan(%)
0,APEX,-58.963583,Turun,0,-6.457342
